In [83]:
import numpy as np
import pickle
import nltk
import pandas as pd

from keras.models import load_model

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from SupportClasses import CleanData

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SmaRt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SmaRt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SmaRt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [84]:
#1 sample
Sample = "@StarIsTheFuture you call @GOPBlackChick house nigger and she's the racist? Fathom the hypocrisy in that statement."
dfsample = pd.DataFrame(data={'Text':[Sample]})

#2 sample
Sample2 = '12 &^£ Welcome to the UK44'
dfsample2 = pd.DataFrame(data={'Text':[Sample2]})

#many samples
filename = 'TESTfile.csv'
df = pd.read_csv(filename, encoding='latin-1')
#df=df.rename(columns = {'Tweet':'Text'})  #Change text column into Text
#df=df.rename(columns = {'class':'label'})  #Change text column into label
print(df)

     Unnamed: 0                                               Text  label
0           707            get fucked racist inbred hillbilly fuck      0
1          2589     lmao lebron expression say nigga fuckin faggot      0
2           748  know ultimate warrior died god love fag anti g...      0
3          1392  oh pain med made brain fuzzy though banjo kazo...      1
4          2058  okay cool one mine black fuzzy pink back spark...      1
..          ...                                                ...    ...
287        2152                                    craving brownie      1
288         781              want represent da life da colored man      1
289        1115  went gold cruz sound upper lower body strength...      1
290        2027  man bitch gon na damn thang u like got say unf...      0
291         131  yes da def da problem teacher spear chucker wo...      1

[292 rows x 3 columns]


In [85]:
#load tfidf
vectorizer = pickle.load(open("vectorizer_Davidson_hatespeech_twitter_data_2classes.pickle", 'rb'))
#load selected features
selectF = pickle.load(open("select_T.pickle", 'rb'))
#load SVM model
ML_model = pickle.load(open("Davidson_hatespeech_twitter_data_2classes_SVM_TFIDF.sav", 'rb'))
#Load CNN model
DL_model = load_model('Davidson_hatespeech_twitter_data_2classes_CNN_TFIDF.h5')

print('All Models Loaded')

All Models Loaded


In [86]:
#clean/process 1 sample
dfsample['cleanedText']=CleanData.cleanAllSample(dfsample['Text'])
print(dfsample)
print('-----')
dfsample2['cleanedText']=CleanData.cleanAllSample(dfsample2['Text'])
print(dfsample2)

                                                Text  \
0  @StarIsTheFuture you call @GOPBlackChick house...   

                                         cleanedText  
0  call house nigger racist fathom hypocrisy stat...  
-----
                         Text cleanedText
0  12 &^£ Welcome to the UK44  welcome uk


In [87]:
#clean/process many samples
df['cleanedText']=CleanData.cleanAllSample(df['Text'])
print(df['cleanedText'])

0                get fucked racist inbred hillbilly fuck
1         lmao lebron expression say nigga fuckin faggot
2      know ultimate warrior died god love fag anti g...
3      oh pain med made brain fuzzy though banjo kazo...
4      okay cool one mine black fuzzy pink back spark...
                             ...                        
287                                      craving brownie
288                want represent da life da colored man
289    went gold cruz sound upper lower body strength...
290    man bitch gon na damn thang u like got say unf...
291    yes da def da problem teacher spear chucker wo...
Name: cleanedText, Length: 292, dtype: object


In [88]:
tfidf = vectorizer.transform(dfsample['cleanedText']).toarray()
Mft = np.concatenate([tfidf],axis=1)
X = selectF.transform(Mft)
y_preds = ML_model.predict(X)
print(y_preds)
y_preds = DL_model.predict_classes(X)
print(y_preds)

[0]
[[0]]


In [91]:
tfidf = vectorizer.transform(dfsample2['cleanedText']).toarray()
Mft = np.concatenate([tfidf],axis=1)
X = selectF.transform(Mft)
y_preds = ML_model.predict(X)
print(y_preds)
y_preds = DL_model.predict_classes(X)
print(y_preds)

[1]
[[1]]


In [92]:
tfidf = vectorizer.transform(df['cleanedText']).toarray()
Mft = np.concatenate([tfidf],axis=1)
y = df['label']
X = selectF.transform(Mft)

In [93]:
y_preds = ML_model.predict(X)
report = classification_report( y, y_preds )
print(report)
print(confusion_matrix(y, y_preds))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96       131
           1       0.96      0.99      0.97       161

    accuracy                           0.97       292
   macro avg       0.97      0.97      0.97       292
weighted avg       0.97      0.97      0.97       292

[[124   7]
 [  2 159]]


In [94]:
y_preds = DL_model.predict_classes(X)
report = classification_report( y, y_preds )
print(report)
print(confusion_matrix(y, y_preds))   

              precision    recall  f1-score   support

           0       0.91      0.73      0.81       131
           1       0.81      0.94      0.87       161

    accuracy                           0.85       292
   macro avg       0.86      0.84      0.84       292
weighted avg       0.85      0.85      0.84       292

[[ 96  35]
 [ 10 151]]
